In [41]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
import scipy

In [42]:
corpus = [
    'Казнить нельзя, помиловать. Нельзя наказывать.',
    'Казнить, нельзя помиловать. Нельзя освободить.',
    'Нельзя не помиловать.',
    'Обязательно освободить.']

In [43]:
#Получаем счетчики слов
TF = CountVectorizer().fit_transform(corpus)

#Строим IDF. К сожалению, в этом задании нам нужно только vectorizer.idf_
#Для стандартных случаев на этой строке все вычисления и заканчиваются.
#Обычно  TFIDF = vectorizer.fit_transform(corpus)
vectorizer = TfidfVectorizer(smooth_idf=False, use_idf=True)
vectorizer.fit_transform(corpus)

## из IDF  в DF
word_doc_freq = 1/np.exp(vectorizer.idf_ - 1)

In [44]:
word_doc_freq, vectorizer.vocabulary_

(array([0.5 , 0.25, 0.25, 0.75, 0.25, 0.5 , 0.75]),
 {'казнить': 0,
  'нельзя': 3,
  'помиловать': 6,
  'наказывать': 1,
  'освободить': 5,
  'не': 2,
  'обязательно': 4})

In [45]:
#TF нормируем и сглаживаем логарифмом (требование задания)
TFIDF = np.log1p(TF/TF.sum(axis=1)) / word_doc_freq


In [46]:
TFIDF.toarray()

array([[0.36464311, 0.72928623, 0.        , 0.44862965, 0.        ,
        0.        , 0.24309541],
       [0.36464311, 0.        , 0.        , 0.44862965, 0.        ,
        0.36464311, 0.24309541],
       [0.        , 0.        , 1.15072829, 0.3835761 , 0.        ,
        0.        , 0.3835761 ],
       [0.        , 0.        , 0.        , 0.        , 1.62186043,
        0.81093022, 0.        ]])

In [47]:
#Масштабируем признаки
# тут задача немного искусственная, поэтому применяю toarray, чтобы нормализовать с вычитанием среднего.
# просто для sparse matrix standardscaler работает вез вычитания среднего
scaledTFIDF = StandardScaler().fit_transform(TFIDF.toarray())

In [48]:
#Домножаем на np.sqrt((4-1)/4) для перевода из DDOF(0) в DDOF(1) для 4 текстов
#(требование задания)
scaledTFIDF *= np.sqrt((len(corpus)-1)/len(corpus))

#Вывод в порядке возрастания DF
for l in scaledTFIDF[:,np.argsort(word_doc_freq)]:
    print (" ".join([ "%.2f" % d for d in l]))

1.50 -0.50 -0.50 0.87 -0.76 0.60 0.16
-0.50 -0.50 -0.50 0.87 0.18 0.60 0.16
-0.50 1.50 -0.50 -0.87 -0.76 0.29 1.04
-0.50 -0.50 1.50 -0.87 1.34 -1.48 -1.36
